In [0]:
%fs ls /FileStore/tables/

In [0]:
file_df = spark.read \
            .format("text") \
            .load("/FileStore/tables/apache-logs.txt")

file_df.printSchema()

In [0]:
display(file_df)

In [0]:
 log_reg = r'^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\S+) "(\S+)" "([^"]*)'

In [0]:
from pyspark.sql.functions import regexp_extract

logs_df = file_df.select(regexp_extract('value', log_reg, 1).alias('ip'),
                         regexp_extract('value', log_reg, 4).alias('date'),
                         regexp_extract('value', log_reg, 6).alias('image'),
                         regexp_extract('value', log_reg, 10).alias('referrer'))

display(logs_df)

In [0]:
logs_df.printSchema()

In [0]:
referrer_df =  logs_df.where("trim(referrer) != '-' ") \
                    .withColumn("referrer", substring_index("referrer", "/", 3)) \
                    .groupBy("referrer") \
                    .count() \
                    .orderBy("count", ascending=False)

display(referrer_df.head(5))